In [6]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import string
import nltk
nltk.download('stopwords')
from Method import * 
import numpy as np
import spacy
import gensim
from gensim import corpora

from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
import ast
from scipy.sparse import csr_matrix

%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
# Bag_of_words
def BoW(data, col):
    '''Return a Vectorizing document using Bag of Words'''
    dictionary = corpora.Dictionary(data[col])
    doc_term_matrix = [dictionary.doc2bow(rev) for rev in data[col]]
    return doc_term_matrix,dictionary

# TF - IDF
def tfidf(data, col):
    '''Return a Vectorizing document using TF IDF'''
    text_strings = [' '.join(tokens) for tokens in data[col]]
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_dtm = tfidf_vectorizer.fit_transform(text_strings)
    # Convert the TF-IDF weighted DTM to a gensim corpus
    corpus = gensim.matutils.Sparse2Corpus(tfidf_dtm, documents_columns=False)

    # Create a dictionary mapping terms to their integer ids
    id2word = dict((v, k) for k, v in tfidf_vectorizer.vocabulary_.items())
    return corpus, id2word


# Word2Vec
def Word2Vec_method(data, col):
    model = Word2Vec(sentences=data[col], vector_size=100, window=5, min_count=1, workers=4)
    
    # Initialize an array to store document vectors
    document_vectors = []
    
    # Iterate over each document
    for sentence_tokens in data[col]:
        # Initialize an array to store word vectors for the current document
        word_vectors = []
        
        # Iterate over each token in the current document
        for token in sentence_tokens:
            if token in model.wv:
                word_vectors.append(model.wv[token])
        
        # If there are word vectors for the current document, compute the average and append it to document_vectors
        if word_vectors:
            document_vector = np.mean(word_vectors, axis=0)  # Compute the average word vector
            document_vectors.append(document_vector)
        else:
            # If no word vectors are found for the current document, append a zero vector
            document_vectors.append(np.zeros(model.vector_size))
        # Flatten the list of vectors
        flat_vectors = [vector for sublist in document_vectors for vector in sublist]
    return flat_vectors

# Bertmodel
def Bertmodel(data, col):
    pass


In [8]:
data_1  = pd.read_csv('Processed_data\data_1.csv', index_col= 0)
data_1['Text_processed'] = data_1['Text_processed'].apply(ast.literal_eval)
data_1.head(2)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Num_words_text,Text_processed
0,231516,B005OOGTBQ,A2SUIKWQTSO3GG,Peter Harrison,0,0,1,1338163200,WAAY overpriced compared to local grocery,"I love Anna's Ginger Thin cookies, and was hop...",75,"[love, anna, ginger, thin, cooky, hoping, find..."
1,384588,B008CTBK7S,A34QMBOSM068W9,Loves to Listen,3,8,1,1305676800,Love this product but the price is rediculous,Don't buy this chocolate PB2 here. Go to Bell ...,67,"[dont, chocolate, bell, plantation, site, four..."


## Bag Of Words

In [9]:
bow = BoW(data_1, 'Text_processed')

# TF- IDF

In [10]:
tfidf = tfidf(data_1, 'Text_processed')
tfidf

<50000x58826 sparse matrix of type '<class 'numpy.float64'>'
	with 1647380 stored elements in Compressed Sparse Row format>